In [1]:
import numpy as np
import pandas as pd



In [2]:
pd.set_option('display.max_rows',6000)

In [60]:
#nous commençons par définir les variables et les initialiser dans la fonction __init__, à l'intérieur de la classe principale. 
#Les variables définies ci-dessous sont des variables d'état ainsi que des compteurs statistiques.

class waiting_file_Simulation:
    def __init__(self): 
        self.clock=0.0                      #simulation clock
        self.event = ''                     # type d'évènement prochain
        self.num_arrivals_A=0                 #number of arrivals A
        self.num_arrivals_B=0                 #number of arrivals B
        self.t_arrival_A=self.gen_int_arr_A()   #heure de la prochaine arrivée d'un client de cathégorie A
        self.t_arrival_B=self.gen_int_arr_B()   #heure de la prochaine arrivée d'un client de cathégorie B
        self.t_departure1=float('inf')      #heure de départ du serveur E1
        self.t_departure2=float('inf')      #heure de départ du serveur E2
        self.dep_sum1=0                     #Somme des temps de service par guishet 1
        self.dep_sum2=0                     #Somme des temps de service par guishet 2
        self.service_time=0.0               #temps de service d'un client 
        self.state_T1=0                     #état actuel du serveur E1 (binaire)
        self.state_T2=0                     #état actuel du serveur E2 (binaire)
        self.num_in_q1=0                    #numéro actuel dans la file d'attente de guichet 1
        self.num_in_q2=0                    #numéro actuel dans la file d'attente de guichet 2
        self.num_of_departures1=0           #nombre de clients servis par guishet 1
        self.num_of_departures2=0           #nombre de clients servis par guishet 2
        self.sum_t_in_system_A =0.0              # temps de séjour d'un client de cathégorie A dans le système **
        self.sum_t_in_system_B =0.0 
        self.sum_t_in_system=0.0           # pour calculer le temps moyenne de séjour d'un client
        self.sum_t_in_line1=0.0             #le temps totale d'attente pour la file 1 
        self.sum_t_in_line2=0.0             #le temps totale d'attente pour la file 2
        self.num_in_system = 0              # le nombre totale de clients pour l'horizone donné
        self.T1_2 = 0                       # indique si le client quitte le systeme ou passe à l'autre serveur 
        self.T2_1 = 0
        self.t_departureB_1 = float('inf')           
        self.t_departureA_2 = float('inf')
        self.type_in_line1 = []
        self.type_in_line2 = []
    def time_adv(self):                                                       
        t_next_event=min(self.t_arrival_A,self.t_arrival_B,self.t_departure1,self.t_departure2,self.t_departureB_1,self.t_departureA_2)  
        self.sum_t_in_line2 += (self.num_in_q2*(t_next_event-self.clock))
        self.sum_t_in_line1 += (self.num_in_q1*(t_next_event-self.clock)) 
        self.clock=t_next_event
        if (self.t_arrival_A == t_next_event)  or (self.t_arrival_B == t_next_event) :

            if self.t_arrival_A<self.t_arrival_B:
                
                self.num_arrivals_A+=1
                self.event = "Arrivée de client de type A"
                self.arrival()
            else :
                self.event = "Arrivée de client de type B"
                self.num_arrivals_B+=1
                self.arrival()
        elif self.t_departure1==t_next_event:
            self.event = "Départ de guichet 1"
            if self.T1_2 == 0 :  
                self.num_arrivals_A -= 1
            self.teller1()
            
                
        
        elif self.t_departure2==t_next_event :
            self.event = "Départ de guichet 2"
            if self.T2_1 == 0:
                self.num_arrivals_B -= 1
            self.teller2()      
            
        elif self.t_departureB_1==t_next_event :
            self.event = "Départ de guichet 1 d'un client de type B"
            self.num_arrivals_B -= 1
            self.teller1()
            self.t_departureB_1 = float('inf')
        else:
            self.event = "Départ de guichet 2 d'un client de type A"
            self.num_arrivals_A -= 1
            self.teller2()
            self.t_departureA_2 = float('inf')
    def arrival(self):
        if self.t_arrival_A < self.t_arrival_B:
            self.num_in_system += 1
           
            if self.num_in_q1 == 0:                                 #schedule next departure or arrival depending on state of servers
                if self.state_T1==1 :
                    self.num_in_q1+=1
                    self.type_in_line1.append("A")
                    self.t_arrival_A=self.clock + self.gen_int_arr_A()
                  
                else:
                    self.state_T1=1
                    self.dep1= self.gen_service_time_teller1()
                    self.service_time = self.dep1
                    self.dep_sum1 += self.dep1
                    self.t_departure1=self.clock + self.dep1
                    self.sum_t_in_system +=self.t_departure1 - self.t_arrival_A 
                    self.t_arrival_A=self.clock+self.gen_int_arr_A()
                    # traitement de cas ou le client se dérige vers le guiChet 2 aprés avoir terminé son service
                    if np.random.choice([0,1])==0:
                        t_arrival = self.t_departure1
                        T1_2 = 1
                        if self.t_arrival_B < self.t_departure1: self.state_T2==1
                        self.t_departure1=float('inf')
                        if self.num_in_q2 ==0:
                            if self.state_T2==1 : 
                                self.num_in_q2+=1
                                self.type_in_line2.append("A")
                            else:
                                self.state_T2=1 
                                self.dep2= self.gen_service_time_teller2()
                                self.dep_sum2 += self.dep2
                                self.service_time = self.dep2
                                self.t_departureA_2= t_arrival + self.dep2 # temps de depart d'un client de type A de guiche        
                        else :
        
                            self.num_in_q2+=1
                            self.type_in_line2.append("A")
                    else :
                        self.T1_2 =0
            else:
                self.num_in_q1+=1
                self.type_in_line1.append("A")
                self.t_arrival_A = self.clock + self.gen_int_arr_A()
        else :
            self.num_in_system += 1
            
            if self.num_in_q2 == 0:                                 #schedule next departure or arrival depending on state of servers
                if self.state_T2==1 :
                    self.num_in_q2+=1
                    self.type_in_line2.append("B")
                    self.t_arrival_B=self.clock+self.gen_int_arr_B()
                  
                else:
                    self.state_T2=1
                    self.dep2= self.gen_service_time_teller2()
                    self.dep_sum2 += self.dep2
                    self.service_time = self.dep2
                    self.t_departure2=self.clock + self.dep2
                    self.sum_t_in_system += self.t_departure2 - self.t_arrival_B
                    self.t_arrival_B=self.clock+self.gen_int_arr_B()
                    # traitement de cas ou le client se dérige vers le guiChet 1 aprés avoir terminé son service
                    if np.random.choice([0,1],p=[0.25,0.75])==0: 
                        t_arrival = self.t_departure2 
                        self.T2_1 = 1
                        if self.t_arrival_A < self.t_departure2: self.state_T1==1
                        if self.num_in_q1 ==0:
                            if self.state_T1==1 :
                                self.num_in_q1+=1
                                self.type_in_line1.append("B")
                            else:
                                self.state_T1=1 
                                self.dep1= self.gen_service_time_teller1()
                                self.service_time = self.dep1
                                self.dep_sum1 += self.dep1
                                self.t_departureB_1= t_arrival + self.dep1 # temps de départ de guichet 1 sachant que il est déja passé 
                                                                                 # par le guichet 2      
                        else :
                            self.num_in_q1+=1  
                            self.type_in_line1.append("B")
                    else :
                        self.T2_1 = 0
                        
            else:
                self.num_in_q2+=1
                self.type_in_line2.append("B")
                self.t_arrival_B=self.clock + self.gen_int_arr_B()
        
    def teller1(self):                #departure from server 1
        self.num_of_departures1 += 1        
        if self.num_in_q1>0:
            self.dep1= self.gen_service_time_teller1()
            self.dep_sum1 += self.dep1
            if self.type_in_line1[0]=="B":    
                self.t_departureB_1=self.clock + self.dep1
            else:
                if np.random.choice([0,1])==0:
                    self.T1_2 = 1
                else :
                    self.T1_2 = 1
                self.t_departure1=self.clock + self.dep1
            self.num_in_q1-=1
            self.type_in_line1.pop(0)
        else:
            self.t_departure1=float('inf') 
            self.state_T1=0                  
    
    def teller2(self):   #departure from server 2
        self.num_of_departures2 += 1  
        if self.num_in_q2>0:
            self.dep2= self.gen_service_time_teller2()
            self.dep_sum2 += self.dep2
            if self.type_in_line2[0]=="A":
                 self.t_departureA_2=self.clock + self.dep2
            else:
                if np.random.choice([0,1],p=[0.25,0.75])==0:
                    self.T2_1 = 1
                else:
                    self.T2_1 = 0
                self.t_departure2=self.clock + self.dep2
            self.num_in_q2-=1
            self.type_in_line2.pop(0)
        else:
            self.t_departure2=float('inf')
            self.state_T2=0          
     
    def gen_int_arr_A(self):                                             #function to generate arrival times using inverse trnasform method
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 1/8)
    
    def gen_int_arr_B(self):                                             #function to generate arrival times using inverse trnasform method
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 1/17) 
    
    def gen_service_time_teller1(self):                                #function to generate service time for teller 1 using inverse trnasform
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 1/20)
    
    def gen_service_time_teller2(self):                                #function to generate service time for teller 1 using inverse trnasform
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 1/30)
    
    
        

In [61]:
s=waiting_file_Simulation()
df=pd.DataFrame(columns=['evenement','temps_arrivée/Départ','num_of_A_in_system','num_of_B_in_system'])
s.__init__()
while s.clock <= 100 : #Horizon de simulation
        s.time_adv()
        a=pd.Series([s.event,s.clock,s.num_arrivals_A,s.num_arrivals_B],index=df.columns)
        df=df.append(a,ignore_index=True)

In [62]:
df

,evenement,temps_arrivée/Départ,num_of_A_in_system,num_of_B_in_system
0,Arrivée de client de type B,0.015321,0,1
1,Départ de guichet 2,0.040042,0,0
2,Arrivée de client de type B,0.136048,0,1
3,Départ de guichet 2,0.137159,0,0
4,Arrivée de client de type B,0.152714,0,1
5,Départ de guichet 2,0.152964,0,0
6,Arrivée de client de type B,0.204802,0,1
7,Départ de guichet 2,0.234375,0,1
8,Arrivée de client de type B,0.237467,0,2
9,Arrivée de client de type B,0.246836,0,3


In [43]:
t = [0]
t[-1]


0